In [6]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table

# define environment before importing any DRP related modules
os.environ["LVMCORE_DIR"] = "/Users/droryn/prog/lvm/lvmcore"
os.environ["LVM_MASTER_DIR"] = "/Users/droryn/work/LVM/data/calib"
os.environ["SAS_BASE_DIR"] = "/Users/droryn/work/LVM/data/sas"
os.environ["LVM_DATA_S"] = "/Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/data/lco"
os.environ["LVM_SPECTRO_REDUX"] = "/Users/droryn/work/LVM/data/redux"
ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

from lvmdrp.functions import imageMethod

MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [7]:
MJD = 60171
print(ORIG_MASTER_DIR)

/Users/droryn/work/LVM/data/calib


In [8]:
print(os.getenv("SAS_BASE_DIR"))
drp.get_frames_metadata(mjd=MJD, overwrite=True)

[INFO]: loading metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5
[INFO]: removing metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5
[INFO]: Creating new metadata store.
[INFO]: going to extract metadata from 180 frames


/Users/droryn/work/LVM/data/sas


extracting metadata: 100%|##########| 180/180 [00:30<00:00,  5.82frame/s]
[INFO]: creating metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5
[INFO]: creating metadata store for tileid = 1111 and mjd = 60171 with 180 new rows
[INFO]: writing raw metadata store to disk
[INFO]: loading metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5
[INFO]: updating metadata store for tileid = 1111 and mjd = 60171 with 0 new rows
[INFO]: final number of rows 180
[INFO]: writing raw metadata store to disk


,hemi,tileid,mjd,rmjd,imagetyp,spec,camera,expnum,exptime,neon,...,xenon,argon,ldls,quartz,quality,qual,stage,status,drpqual,name
0,s,1111,60171,60171,dark,sp2,r2,3109,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-r2-00003109.fits
1,s,1111,60171,60171,dark,sp1,r1,3098,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-r1-00003098.fits
2,s,1111,60171,60171,dark,sp2,z2,3098,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003098.fits
3,s,1111,60171,60171,dark,sp1,z1,3109,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-z1-00003109.fits
4,s,1111,60171,60171,dark,sp3,b3,3099,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-b3-00003099.fits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,s,1111,60171,60171,dark,sp2,z2,3111,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003111.fits
176,s,1111,60171,60171,dark,sp3,r3,3116,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-r3-00003116.fits
177,s,1111,60171,60171,dark,sp3,r3,3106,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-r3-00003106.fits
178,s,1111,60171,60171,dark,sp1,r1,3111,5.0,False,...,False,False,False,False,excellent,0,1,0,0,sdR-s-r1-00003111.fits


In [9]:
frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("imagetyp == 'dark'", inplace=True)
frames_table.sort_values("camera", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5
[INFO]: found 180 frames in store '/Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60171/raw_metadata.hdf5'
[INFO]: number of frames after filtering 180
[INFO]: total number of frames found 180


In [10]:
#os.listdir(ORIG_MASTER_DIR)
masters_mjd = qdrp.get_master_mjd(MJD)
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
print(masters_path)

/Users/droryn/work/LVM/data/calib/60142


In [12]:

# define target folder for detrended pixelflats
target_dir = "/Users/droryn/work/LVM/data/redux/pixflats"
os.makedirs(target_dir, exist_ok=True)

dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    flat_path = path.full("lvm_raw", camspec=pixflat["camera"], **pixflat)
    # print(flat_path)
    
    # output path
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")
    dflat_paths.append(dflat_path)
    
    # calib paths
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{cam}.fits")
    
    imageMethod.preproc_raw_frame(in_image=flat_path, out_image=dflat_path, replace_with_nan=False)
    imageMethod.detrend_frame(in_image=dflat_path, out_image=dflat_path, in_bias=mbias_path, reject_cr=False, replace_with_nan=False)


[INFO]: starting preprocessing of raw image 'sdR-s-b1-00003106.fits.gz'


[INFO]: using header IMAGETYP = 'dark'
[INFO]: exposure time 5.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 985.00 +/- 17.02 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 990.00 +/- 15.94 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 995.00 +/- 16.34 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 995.00 +/- 15.45 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 1402 (0.0084 %) pixels masked
[INFO]: writing preprocessed image to lvm-dpixflat-b1-3106.fits
[INFO]: plotting results
[INFO]: t

In [13]:
frames_table["dflat_path"] = dflat_paths

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:
    dflat_paths_cam = cam_groups.get_group(cam)["dflat_path"]
    
    # define output combined pixelflat path
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    
    imageMethod.create_master_frame(in_images=dflat_paths_cam, out_image=mflat_path)

[INFO]: input frames: /Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3106.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3111.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3114.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3108.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3104.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3098.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3115.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3103.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3113.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3116.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3110.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3100.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3101.fits,/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3105.fits,/Users/drory

In [20]:
from scipy import ndimage as ndi
from astropy.io import fits

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:    
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    pixflat_path = os.path.join(target_dir, f"lvm-cpixflat-{cam}.fits")

    with fits.open(mflat_path) as hdul:
        print("Reading :", mflat_path)
        image = hdul[0].data
        filtered = ndi.median_filter(image, size=20, mode='nearest')
        flat = image/filtered
        out = fits.HDUList(fits.PrimaryHDU(flat))
        print("Writing :", pixflat_path)
        out.writeto(pixflat_path)
        out.close()


Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b1.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b1.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b2.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b2.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b3.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b3.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-r1.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-r1.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-r2.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-r2.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-r3.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-r3.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-z1.fits
Writing : /Users/droryn/work/LVM/data/